### Connect to Azure Machine Learning workspace

In [73]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
client = MLClient.from_config(credential)


Found the config file in: .\config.json


### Get Credentials from Key Vault

In [74]:
from azure.keyvault.secrets import SecretClient

keyvault_uri = "https://plato-default-key-vault.vault.azure.net/"
secret_client = SecretClient(vault_url=keyvault_uri, credential=credential)

tenant_id = secret_client.get_secret('azure-tenant-id').value
client_id = secret_client.get_secret('ml-client-id').value
client_secret = secret_client.get_secret('ml-client-secret').value

### Configure the Command

### CUDA Train Job

In [76]:
from azure.ai.ml import command, Input, Output, UserIdentityConfiguration
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes

identity = UserIdentityConfiguration()
registered_model_name = "plato-gpt-trained-model"

input_path = 'azureml://subscriptions/af9d95ee-d424-470c-8f1a-3540102fef9f/resourcegroups/plato/workspaces/plato-workspace/datastores/workspaceblobstore/paths/sam_harris_dataset/data.txt'
input_data_type = AssetTypes.URI_FILE
input_mode = InputOutputModes.RO_MOUNT

config_path = './config.json'
config_file_type = AssetTypes.URI_FILE
config_file_mode = InputOutputModes.RO_MOUNT

output_path = 'azureml://subscriptions/af9d95ee-d424-470c-8f1a-3540102fef9f/resourcegroups/plato/workspaces/plato-workspace/datastores/workspaceblobstore/paths/azureml/'
output_data_type = AssetTypes.URI_FOLDER
output_mode = InputOutputModes.RW_MOUNT

env = 'azureml:Plato-GPT-env:4'


# configure the command job
cuda_job = command(
    inputs=dict(
        data=Input(
            type=input_data_type,
            path=input_path,
            mode=input_mode
        ),
        config=Input(
            type=config_file_type,
            path=config_path,
            mode=config_file_mode
        ), 
        start_fresh = True,
        tokenizer_type = 'nltk',

        batch_size = 16,
        block_size = 256,
        max_epoch = 5000,
        n_embd = 768,
        n_head = 12,
        n_layer = 14,
        warmup_iters = 400,
        lr_decay_iters = 4500,
    ),
    outputs=dict(
        log_out=Output(
            type=output_data_type,
            path=output_path,
            output_mode=output_mode
        ),
    ),
    environment_variables= {
        'AZURE_TENANT_ID': tenant_id,
        'AZURE_CLIENT_ID': client_id,
        'AZURE_CLIENT_SECRET': client_secret,
    },
    code="./src",  # location of source code
    command="python train.py \
            --data ${{inputs.data}} \
            --config ${{inputs.config}} \
            --start_fresh ${{inputs.start_fresh}} \
            --tokenizer_type ${{inputs.tokenizer_type}} \
            --max_epoch ${{inputs.max_epoch}} \
            --batch_size ${{inputs.batch_size}} \
            --block_size ${{inputs.block_size}} \
            --n_embd ${{inputs.n_embd}} \
            --n_head ${{inputs.n_head}} \
            --n_layer ${{inputs.n_layer}} \
            --warmup_iters ${{inputs.warmup_iters}} \
            --lr_decay_iters ${{inputs.lr_decay_iters}}",
    environment=env,
    compute='nv12',
    experiment_name="Plato-GPT-2-NLTK",
    display_name="NLTK-Encoding-based-cuda",
    identity=identity
)

In [77]:
client.create_or_update(cuda_job)

Experiment,Name,Type,Status,Details Page
Plato-GPT-2-NLTK,loving_flower_5ydt4prb7x,command,Starting,Link to Azure Machine Learning studio


### CPU Train Job ( Small )

In [ ]:
from azure.ai.ml import command, Input, Output, UserIdentityConfiguration
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes

identity = UserIdentityConfiguration()
registered_model_name = "plato-gpt-trained-model"

input_path = 'azureml://subscriptions/af9d95ee-d424-470c-8f1a-3540102fef9f/resourcegroups/plato/workspaces/plato-workspace/datastores/workspaceblobstore/paths/sam_harris_dataset/data.txt'
input_data_type = AssetTypes.URI_FILE
input_mode = InputOutputModes.RO_MOUNT

config_path = './config.json'
config_file_type = AssetTypes.URI_FILE
config_file_mode = InputOutputModes.RO_MOUNT

output_path = 'azureml://subscriptions/af9d95ee-d424-470c-8f1a-3540102fef9f/resourcegroups/plato/workspaces/plato-workspace/datastores/workspaceblobstore/paths/azureml/'
output_data_type = AssetTypes.URI_FOLDER
output_mode = InputOutputModes.RW_MOUNT

env = 'azureml:Plato-GPT-env:4'


# configure the command job
cpu_small_job = command(
    inputs=dict(
        data=Input(
            type=input_data_type,
            path=input_path,
            mode=input_mode
        ),
        config=Input(
            type=config_file_type,
            path=config_path,
            mode=config_file_mode
        ), 
        start_fresh = True,
        tokenizer_type = 'nltk',

        batch_size = 16,
        block_size = 512,
        max_epoch = 3000,
        n_embd = 768,
        n_head = 12,
        n_layer = 14,
        warmup_iters = 200,
        lr_decay_iters = 2500,
    ),
    outputs=dict(
        log_out=Output(
            type=output_data_type,
            path=output_path,
            output_mode=output_mode
        ),
    ),
    environment_variables= {
        'AZURE_TENANT_ID': tenant_id,
        'AZURE_CLIENT_ID': client_id,
        'AZURE_CLIENT_SECRET': client_secret,
    },
    code="./src",  # location of source code
    command="python train.py \
            --data ${{inputs.data}} \
            --config ${{inputs.config}} \
            --start_fresh ${{inputs.start_fresh}} \
            --tokenizer_type ${{inputs.tokenizer_type}} \
            --max_epoch ${{inputs.max_epoch}} \
            --batch_size ${{inputs.batch_size}} \
            --block_size ${{inputs.block_size}} \
            --n_embd ${{inputs.n_embd}} \
            --n_head ${{inputs.n_head}} \
            --n_layer ${{inputs.n_layer}} \
            --warmup_iters ${{inputs.warmup_iters}} \
            --lr_decay_iters ${{inputs.lr_decay_iters}}",
    environment=env,
    compute='CPU-Cluster',
    experiment_name="Plato-GPT-2-NLTK",
    display_name="NLTK-Encoding-based-cpu-S",
    identity=identity
)

In [ ]:
client.create_or_update(cpu_small_job)

### CPU Train Job ( Large )

In [ ]:
from azure.ai.ml import command, Input, Output, UserIdentityConfiguration
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes

identity = UserIdentityConfiguration()
registered_model_name = "plato-gpt-trained-model"

input_path = 'azureml://subscriptions/af9d95ee-d424-470c-8f1a-3540102fef9f/resourcegroups/plato/workspaces/plato-workspace/datastores/workspaceblobstore/paths/sam_harris_dataset/data.txt'
input_data_type = AssetTypes.URI_FILE
input_mode = InputOutputModes.RO_MOUNT

config_path = './config.json'
config_file_type = AssetTypes.URI_FILE
config_file_mode = InputOutputModes.RO_MOUNT

output_path = 'azureml://subscriptions/af9d95ee-d424-470c-8f1a-3540102fef9f/resourcegroups/plato/workspaces/plato-workspace/datastores/workspaceblobstore/paths/azureml/'
output_data_type = AssetTypes.URI_FOLDER
output_mode = InputOutputModes.RW_MOUNT

env = 'azureml:Plato-GPT-env:4'


# configure the command job
cpu_large_job = command(
    inputs=dict(
        data=Input(
            type=input_data_type,
            path=input_path,
            mode=input_mode
        ),
        config=Input(
            type=config_file_type,
            path=config_path,
            mode=config_file_mode
        ), 
        start_fresh = True,
        tokenizer_type = 'nltk',

        batch_size = 16,
        block_size = 512,
        max_epoch = 8000,
        n_embd = 768,
        n_head = 12,
        n_layer = 14,
        warmup_iters = 500,
        lr_decay_iters = 7500,
    ),
    outputs=dict(
        log_out=Output(
            type=output_data_type,
            path=output_path,
            output_mode=output_mode
        ),
    ),
    environment_variables= {
        'AZURE_TENANT_ID': tenant_id,
        'AZURE_CLIENT_ID': client_id,
        'AZURE_CLIENT_SECRET': client_secret,
    },
    code="./src",  # location of source code
    command="python train.py \
            --data ${{inputs.data}} \
            --config ${{inputs.config}} \
            --start_fresh ${{inputs.start_fresh}} \
            --tokenizer_type ${{inputs.tokenizer_type}} \
            --max_epoch ${{inputs.max_epoch}} \
            --batch_size ${{inputs.batch_size}} \
            --block_size ${{inputs.block_size}} \
            --n_embd ${{inputs.n_embd}} \
            --n_head ${{inputs.n_head}} \
            --n_layer ${{inputs.n_layer}} \
            --warmup_iters ${{inputs.warmup_iters}} \
            --lr_decay_iters ${{inputs.lr_decay_iters}}",
    environment=env,
    compute='CPU-Cluster',
    experiment_name="Plato-GPT-2-NLTK",
    display_name="NLTK-Encoding-based-cpu-L",
    identity=identity
)

In [ ]:
client.create_or_update(cpu_large_job)